# Avaliação Automatizada de Suturas Cirúrgicas com Deep Learning

### Import das Bibliotecas

In [366]:
import sys
print(sys.executable)

import torch
print(torch.__version__)
print(torch.cuda.is_available())

c:\Users\35193\miniconda3\envs\ap_torch\python.exe
2.7.0+cu118
True


In [367]:
import os
import numpy as np
import pandas as pd
import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2.7.0+cu118
True
NVIDIA GeForce RTX 2060


### Verificar CUDA

In [368]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

2.7.0+cu118
True
NVIDIA GeForce RTX 2060


### Configuração Inicial

In [369]:
VIDEO_DIR = r"../../OSS_dataset/Train/videos"             # Pasta com vídeos cirúrgicos
SAIDA_DIR = r"frames_limpos_model1"      # Pasta para guardar o último frame limpo de cada vídeo
EXCEL_PATH = r"../../OSS_dataset/Train/OSATS.xlsx"    # Excel de classificação
CSV_PATH = r"../../OSS_dataset/Train/OSATS.csv"
FRAME_INTERVAL = 10                            # Ex: 30 = 1 frame por segundo (assumindo 30 fps)
FRAME_SIZE = (224, 224)                        # Tamanho padrão para input da CNN

os.makedirs(SAIDA_DIR, exist_ok=True)

### Dataset PyTorch personalizado

In [370]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### Carregar dfs do ficheiro csv

In [371]:
# Carregar ficheiros
# train_df = pd.read_csv("dataframes/train_df.csv")
# test_df = pd.read_csv("dataframes/test_df.csv")

train_df = pd.read_csv("dataframes_por_inspetor/train_C.csv")
test_df = pd.read_csv("dataframes_por_inspetor/test_C.csv")

# Target
y_train = train_df["GLOBA_RATING_SCORE"]

# Colunas que não devem ser usadas como input (identificadores, alvos, anotações humanas)
colunas_a_excluir = [
    'video','GLOBA_RATING_SCORE',
    'OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT', 'OSATS_SUTURE',
    'OSATS_FLOW', 'OSATS_KNOWLEDGE', 'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY'
]

# Inferir automaticamente todas as features disponíveis
features = [col for col in train_df.columns if col not in colunas_a_excluir]

# Normalização
temp_X = train_df[features]
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(temp_X), columns=features)
X_test = pd.DataFrame(scaler.transform(test_df[features]), columns=features)

# Validação holdout
# X_train_part, X_val, y_train_part, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=42)

### Método para passar de classificação real para classe

In [372]:
# Mapear os valores reais de GRS para classes 0–3

def classificar_grs(grs):
    if grs <= 15:
        return 0
    elif grs <= 23:
        return 1
    elif grs <= 31:
        return 2
    else:
        return 3

# Modelação

In [373]:
# # Dataset e dataloaders
# train_dataset = TabularDataset(X_train_part, y_train_part)
# val_dataset = TabularDataset(X_val, y_val)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

train_dataset = TabularDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## Modelo MLP para regressão

In [374]:
# Modelo MLP para regressão
class MLPRegressor(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128, 64]):
        super(MLPRegressor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_sizes[1], 1)  # regressão
        )

    def forward(self, x):
        return self.network(x).squeeze()

model = MLPRegressor(input_size=X_train.shape[1])
# criterion = nn.MSELoss()
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Treino com validação

In [375]:
model.train()
for epoch in range(10):  # ajusta conforme necessário
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 69.1822
Epoch 2, Loss: 67.7172
Epoch 3, Loss: 65.2037
Epoch 4, Loss: 64.7559
Epoch 5, Loss: 62.0965
Epoch 6, Loss: 61.1170
Epoch 7, Loss: 56.8387
Epoch 8, Loss: 53.1097
Epoch 9, Loss: 47.5896
Epoch 10, Loss: 42.3903


### Fazer previsões

In [376]:
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor).numpy()

# Clip para [8, 40]
grs_pred_val = np.clip(preds, 8, 40)

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

### Gerar csv de submissão

In [377]:
df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_mlp_reg.csv", index=False)
print("✅ CSV gerado: task1_predicoes_mlp_reg.csv")

✅ CSV gerado: task1_predicoes_mlp_reg.csv


### Avaliar Modelo

In [378]:
df_osats = pd.read_csv(CSV_PATH, sep=';')
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)
df_pred = pd.read_csv("task1_predicoes_mlp_reg.csv")
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]

y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")


📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.1230
📈 Accuracy: 0.3263
💸 Expected Cost: 1.2421


## Modelo MLP mais complexo

In [379]:
class MLPRegressor(nn.Module):
    def __init__(self, input_size):
        super(MLPRegressor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.network(x).squeeze()

### Treinar durante mais épocas com early stopping

In [380]:
best_loss = float('inf') 

model.train()
for epoch in range(100):  # ajusta conforme necessário
    total_loss = 0
    for inputs, targets in train_loader:
        targets = targets.squeeze() 
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
    if total_loss < best_loss:
        best_loss = total_loss
        torch.save(model.state_dict(), "best_model1.pt")


Epoch 1, Loss: 36.4587
Epoch 2, Loss: 30.1844
Epoch 3, Loss: 24.3135
Epoch 4, Loss: 22.4631
Epoch 5, Loss: 20.2439
Epoch 6, Loss: 17.7867
Epoch 7, Loss: 15.8951
Epoch 8, Loss: 15.1653
Epoch 9, Loss: 12.8141
Epoch 10, Loss: 12.2422
Epoch 11, Loss: 10.6552
Epoch 12, Loss: 10.8403
Epoch 13, Loss: 8.5511
Epoch 14, Loss: 9.4721
Epoch 15, Loss: 8.8070
Epoch 16, Loss: 8.9243
Epoch 17, Loss: 7.6299
Epoch 18, Loss: 8.7338
Epoch 19, Loss: 8.0635
Epoch 20, Loss: 9.4353
Epoch 21, Loss: 8.1003
Epoch 22, Loss: 7.3670
Epoch 23, Loss: 8.1157
Epoch 24, Loss: 8.2070
Epoch 25, Loss: 6.9734
Epoch 26, Loss: 7.1878
Epoch 27, Loss: 7.4072
Epoch 28, Loss: 7.1977
Epoch 29, Loss: 7.8427
Epoch 30, Loss: 8.7588
Epoch 31, Loss: 7.1717
Epoch 32, Loss: 7.5988
Epoch 33, Loss: 6.5959
Epoch 34, Loss: 7.6922
Epoch 35, Loss: 6.5979
Epoch 36, Loss: 6.5502
Epoch 37, Loss: 6.9044
Epoch 38, Loss: 8.4817
Epoch 39, Loss: 7.4474
Epoch 40, Loss: 6.4347
Epoch 41, Loss: 7.3064
Epoch 42, Loss: 6.5341
Epoch 43, Loss: 6.7157
Epoch 44

### Usar melhor modelo para inferência 

In [381]:
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
model.load_state_dict(torch.load("best_model1.pt"))
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor).numpy()

### Avaliar Modelo

In [382]:
# Clip para [8, 40]
grs_pred_val = np.clip(preds, 8, 40)

grs_pred_classe = [classificar_grs(v) for v in grs_pred_val]

df_submissao = pd.DataFrame({
    'VIDEO': test_df['video'],
    'GRS': grs_pred_classe
})
df_submissao.to_csv("task1_predicoes_mlp2_reg.csv", index=False)
print("✅ CSV gerado: task1_predicoes_mlp2_reg.csv")

df_osats = pd.read_csv(CSV_PATH, sep=';')
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)
df_pred = pd.read_csv("task1_predicoes_mlp2_reg.csv")
df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")

df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]

y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("📊 Avaliação baseada nas anotações reais (média por vídeo):")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")


✅ CSV gerado: task1_predicoes_mlp2_reg.csv
📊 Avaliação baseada nas anotações reais (média por vídeo):
🎯 F1-Score (macro): 0.3378
📈 Accuracy: 0.4526
💸 Expected Cost: 0.6105


In [383]:
# ✅ CSV gerado: task1_predicoes_mlp2_reg.csv
# 📊 Avaliação baseada nas anotações reais (média por vídeo):
# 🎯 F1-Score (macro): 0.6662
# 📈 Accuracy: 0.7032
# 💸 Expected Cost: 0.3004

## Modelo MLP otimizado 

In [384]:
class MLPRegressor(nn.Module):
    def __init__(self, input_size):
        super(MLPRegressor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.network(x).squeeze()

model = MLPRegressor(input_size=X_train.shape[1]).to(device)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

### Treino com validação

In [385]:
best_loss = float('inf')
model.train()
for epoch in range(100):
    total_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        targets = targets.squeeze()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
    if total_loss < best_loss:
        best_loss = total_loss
        torch.save(model.state_dict(), "best_model2.pt")


Epoch 1, Loss: 67.7602
Epoch 2, Loss: 64.3203
Epoch 3, Loss: 63.3486
Epoch 4, Loss: 60.0214
Epoch 5, Loss: 58.2115
Epoch 6, Loss: 55.1215
Epoch 7, Loss: 51.4848
Epoch 8, Loss: 44.1978
Epoch 9, Loss: 41.5949
Epoch 10, Loss: 36.2811
Epoch 11, Loss: 29.9561
Epoch 12, Loss: 25.1380
Epoch 13, Loss: 20.9025
Epoch 14, Loss: 15.8180
Epoch 15, Loss: 13.1460
Epoch 16, Loss: 12.4138
Epoch 17, Loss: 11.9700
Epoch 18, Loss: 11.3955
Epoch 19, Loss: 9.5998
Epoch 20, Loss: 9.7475
Epoch 21, Loss: 11.2690
Epoch 22, Loss: 9.8160
Epoch 23, Loss: 8.5150
Epoch 24, Loss: 8.9882
Epoch 25, Loss: 8.5698
Epoch 26, Loss: 8.0946
Epoch 27, Loss: 7.7505
Epoch 28, Loss: 9.7088
Epoch 29, Loss: 8.1728
Epoch 30, Loss: 7.9086
Epoch 31, Loss: 7.6220
Epoch 32, Loss: 9.0900
Epoch 33, Loss: 7.6221
Epoch 34, Loss: 7.6883
Epoch 35, Loss: 7.5945
Epoch 36, Loss: 7.1809
Epoch 37, Loss: 8.6807
Epoch 38, Loss: 8.6375
Epoch 39, Loss: 8.2227
Epoch 40, Loss: 7.1510
Epoch 41, Loss: 7.2973
Epoch 42, Loss: 7.6233
Epoch 43, Loss: 8.4611
E

### Previsão

In [386]:
model.load_state_dict(torch.load("best_model2.pt"))
model.eval()
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
with torch.no_grad():
    preds = model(X_test_tensor).cpu().numpy()

grss = np.clip(preds, 8, 40)
grss_classe = [classificar_grs(v) for v in grss]

### Guardar em csv

In [387]:
sub = pd.DataFrame({"VIDEO": test_df["video"], "GRS": grss_classe})
sub.to_csv("task1_predicoes_mlp_final.csv", index=False)
print("✅ CSV gerado: task1_predicoes_mlp_final.csv")

✅ CSV gerado: task1_predicoes_mlp_final.csv


### Avaliação

In [388]:
df_osats = pd.read_csv(CSV_PATH, sep=';')
df_media_grs = df_osats.groupby("VIDEO")["GLOBA_RATING_SCORE"].mean().reset_index()
df_media_grs.rename(columns={"GLOBA_RATING_SCORE": "GRS_REAL"}, inplace=True)
df_pred = sub

df_avaliacao = pd.merge(df_pred, df_media_grs, on="VIDEO", how="inner")
df_avaliacao["GRS_REAL_CLASS"] = df_avaliacao["GRS_REAL"].apply(classificar_grs)
df_avaliacao["GRS_PRED_CLASS"] = df_avaliacao["GRS"]

y_true = df_avaliacao["GRS_REAL_CLASS"]
y_pred = df_avaliacao["GRS_PRED_CLASS"]

f1 = f1_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
cost = np.mean(np.abs(y_true - y_pred))

print("\n📊 Avaliação com modelo final confirmado:")
print(f"🎯 F1-Score (macro): {f1:.4f}")
print(f"📈 Accuracy: {acc:.4f}")
print(f"💸 Expected Cost: {cost:.4f}")



📊 Avaliação com modelo final confirmado:
🎯 F1-Score (macro): 0.3042
📈 Accuracy: 0.4526
💸 Expected Cost: 0.7579


## Tabela comparativa entre modelos e datasets

In [389]:
import pandas as pd

# Tabela de resultados completa (geral + A + B + C)
resultados = [
    {"modelo": "MLP1", 
     "geral": "F1: 0.5748 | Acc: 0.6396 | Cost: 0.3675", 
     "A": "F1: 0.1289 | Acc: 0.3474 | Cost: 1.2211", 
     "B": "F1: 0.1289 | Acc: 0.3474 | Cost: 1.2211", 
     "C": "F1: 0.1260 | Acc: 0.3368 | Cost: 1.2316"},
     
    {"modelo": "MLP2", 
     "geral": "F1: 0.6662 | Acc: 0.7032 | Cost: 0.3004", 
     "A": "F1: 0.4393 | Acc: 0.4632 | Cost: 0.5895", 
     "B": "F1: 0.4973 | Acc: 0.5789 | Cost: 0.4947", 
     "C": "F1: 0.4431 | Acc: 0.4737 | Cost: 0.5895"},
     
    {"modelo": "MLP3", 
     "geral": "F1: 0.5975 | Acc: 0.6608 | Cost: 0.3428", 
     "A": "F1: 0.4013 | Acc: 0.5053 | Cost: 0.6632", 
     "B": "F1: 0.4572 | Acc: 0.4316 | Cost: 0.6105", 
     "C": "F1: 0.4431 | Acc: 0.4737 | Cost: 0.5895"},
]

df = pd.DataFrame(resultados)
print(df.to_string(index=False))


modelo                                   geral                                       A                                       B                                       C
  MLP1 F1: 0.5748 | Acc: 0.6396 | Cost: 0.3675 F1: 0.1289 | Acc: 0.3474 | Cost: 1.2211 F1: 0.1289 | Acc: 0.3474 | Cost: 1.2211 F1: 0.1260 | Acc: 0.3368 | Cost: 1.2316
  MLP2 F1: 0.6662 | Acc: 0.7032 | Cost: 0.3004 F1: 0.4393 | Acc: 0.4632 | Cost: 0.5895 F1: 0.4973 | Acc: 0.5789 | Cost: 0.4947 F1: 0.4431 | Acc: 0.4737 | Cost: 0.5895
  MLP3 F1: 0.5975 | Acc: 0.6608 | Cost: 0.3428 F1: 0.4013 | Acc: 0.5053 | Cost: 0.6632 F1: 0.4572 | Acc: 0.4316 | Cost: 0.6105 F1: 0.4431 | Acc: 0.4737 | Cost: 0.5895
